# Basic classification

옷, 신발, 셔츠 같은 이미지를 분류하기 위한 튜토리얼으로 tensorflow 를 훝어보기 위한 것이다. 자세한 설명은 여기서는 다루지 않는다.

In [2]:
# Tensorflow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.11.0


### Import the Fashion MNIST

여기서는 [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist)을 사용하며, 70,000 개의 grayscale 이미지(28 x 28 pixels)와 10개의 카테고리로 이루어진다. 

### Reference

https://www.tensorflow.org/tutorials/keras/basic_classification